In [ ]:
from os.path import join, abspath, dirname

import sys
sys.path.insert(0, abspath(join(dirname("."), '..')))

from modules.helpers.constants import PERCENTAGE_DICT
import os
import json
import pandas as pd
import math

In [ ]:
# this finds our json files
path_to_json = '../results/'
json_files = [pos_json for pos_json in os.listdir(
    path_to_json) if pos_json.endswith('.json')]

In [ ]:
# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = []
json_blocked_data = []

# we need both the json and an index number so use enumerate()
for index, file_name in enumerate(json_files):
    tokens = file_name.replace('.json', '').split('_')
    algorithm = tokens[1]
    nodes_to_block = int(tokens[2])
    graph_stats = tokens[3].split(']')
    num_nodes = int(graph_stats[0].replace('[', ''))
    num_edges = int(graph_stats[1].replace('[', ''))
    percentile_name = tokens[4]
    percentile = PERCENTAGE_DICT[percentile_name]
    num_seeds = num_nodes * percentile * 0.01

    with open(os.path.join(path_to_json, file_name)) as json_file:
        json_text = json.load(json_file)

        jsons_data.append([index, num_nodes, num_edges, json_text['Total time'],
                          nodes_to_block, num_seeds, algorithm, percentile_name])
        json_blocked_data.append([index, json_text['Blocked nodes']])

jsons_df = pd.DataFrame(jsons_data, columns=[
                        'id', 'num_nodes', 'num_edges', 'total_time', 'nodes_to_block', 'num_seeds', 'algorithm', 'percentile_name'])
json_blocked_df = pd.DataFrame(json_blocked_data, columns=['id', 'nodes'])

In [ ]:
jsons_df.to_csv(join(path_to_json, 'simulationData.csv'), index=False)
json_blocked_df.to_csv(join(path_to_json, 'simulationNodes.csv'), index=False)

In [ ]:
idx = 0
path_to_simulations = '../results/simulations'
json_files = [pos_json for pos_json in os.listdir(
    path_to_simulations) if pos_json.endswith('.json')]
sim_data = pd.DataFrame(columns=['simulation_index', 'mean_activated',
                        'var_activated', 'mean_saved', 'var_saved', 'saved_nodes', 'activated_nodes'])
solvers_key = 'solvers'

In [ ]:
for _, file_name in enumerate(json_files):
    with open(os.path.join(path_to_simulations, file_name)) as json_file:
        json_text = json.load(json_file)

        for sim_json in json_text:
            if solvers_key in sim_json:
                solvers_data = sim_json[solvers_key]
                for sim_id in solvers_data.keys():
                    mean_activated = solvers_data[sim_id]['activated nodes']['mean']
                    var_activated = solvers_data[sim_id]['activated nodes']['var']
                    mean_saved = solvers_data[sim_id]['saved nodes']['mean']
                    var_saved = solvers_data[sim_id]['saved nodes']['var']

                    sim_data.loc[idx] = [sim_id, mean_activated, var_activated, mean_saved, mean_activated, int(
                        math.ceil(mean_saved)), int(math.ceil(mean_activated))]
                    idx += 1
            else:
                continue


In [30]:
sim_data.to_csv(join(path_to_json, 'simulationStats.csv'), index=False)